## BIBIT

In [14]:
from google_play_scraper import Sort, reviews_all
import pymongo

def scrape_google_play_reviews(package_name, collection_name, num_loops):
    # Koneksi ke server MongoDB
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client["google_play_store_data"]
    collection = db[collection_name]

    # Looping sebanyak num_loops
    for _ in range(num_loops):
        # Looping untuk rating dari 1 hingga 5
        for rating in range(1, 6):
            # Looping untuk Sort.NEWEST dan Sort.MOST_RELEVANT
            for sort_option in [Sort.NEWEST, Sort.MOST_RELEVANT]:
                # Scrape semua ulasan yang tersedia
                result = reviews_all(
                    package_name,
                    sleep_milliseconds=0, # defaultnya 0
                    lang='id', # defaultnya 'en'
                    country='id', # defaultnya 'us'
                    count=500,
                    filter_score_with=rating,
                    sort=sort_option,
                )

                # Mencari konten ulasan yang sudah ada di MongoDB
                existing_contents = collection.distinct('reviewId')

                # Memfilter ulasan baru yang tidak duplikat berdasarkan konten
                new_unique_reviews = [review for review in result if review['reviewId'] not in existing_contents]

                # Menyimpan ulasan baru yang tidak duplikat
                if new_unique_reviews:
                    collection.insert_many(new_unique_reviews)
                    print(f"{len(new_unique_reviews)} ulasan baru dengan konten unik, rating {rating}, dan sort {sort_option.name} telah disimpan ke dalam MongoDB.")
                else:
                    print(f"Tidak ada ulasan baru dengan konten unik, rating {rating}, dan sort {sort_option.name} yang perlu disimpan.")

    print("Proses scraping selesai.")

# Input package name aplikasi yang akan di-scrape ulasannya
package_name = 'com.bibit.bibitid'
collection_name = 'raw_data_bibit'

num_loops = 5  # Jumlah loop yang diinginkan

# Memanggil fungsi untuk melakukan scraping
scrape_google_play_reviews(package_name, collection_name, num_loops)


Tidak ada ulasan baru dengan konten unik, rating 1, dan sort NEWEST yang perlu disimpan.
356 ulasan baru dengan konten unik, rating 1, dan sort MOST_RELEVANT telah disimpan ke dalam MongoDB.
Tidak ada ulasan baru dengan konten unik, rating 2, dan sort NEWEST yang perlu disimpan.
19 ulasan baru dengan konten unik, rating 2, dan sort MOST_RELEVANT telah disimpan ke dalam MongoDB.
Tidak ada ulasan baru dengan konten unik, rating 3, dan sort NEWEST yang perlu disimpan.
209 ulasan baru dengan konten unik, rating 3, dan sort MOST_RELEVANT telah disimpan ke dalam MongoDB.
Tidak ada ulasan baru dengan konten unik, rating 4, dan sort NEWEST yang perlu disimpan.
376 ulasan baru dengan konten unik, rating 4, dan sort MOST_RELEVANT telah disimpan ke dalam MongoDB.
Tidak ada ulasan baru dengan konten unik, rating 5, dan sort NEWEST yang perlu disimpan.
278 ulasan baru dengan konten unik, rating 5, dan sort MOST_RELEVANT telah disimpan ke dalam MongoDB.
Tidak ada ulasan baru dengan konten unik, rati

In [15]:
import pandas as pd
from pymongo import MongoClient

# Menghubungkan ke server MongoDB (secara lokal)
client = MongoClient('localhost', 27017)

# Memilih database yang akan digunakan (jika tidak ada, akan dibuat)
db = client['google_play_store_data']

# Memilih koleksi dalam database (jika tidak ada, akan dibuat)
collection = db['raw_data_bibit']  # Mengganti 'nama_koleksi' dengan 'product_reviews'

# Mengambil semua dokumen dalam koleksi
cursor = collection.find()

# Mengonversi data ke dalam bentuk DataFrame
df_bibit = pd.DataFrame(list(cursor))

# Menutup koneksi dengan server MongoDB
client.close()


df_bibit


,_id,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,6638614cdd520c9ba22ee66c,ed2d58b8-16cb-4ea7-ba6d-5776aea3fd3a,adam takur,https://play-lh.googleusercontent.com/a/ACg8oc...,Mau daftar aj di persulit dari bulan April gag...,1,0,None,2024-05-05 10:54:32,None,NaT,None
1,6638614cdd520c9ba22ee66d,d0984278-c286-42b8-bf73-e71e45bec63e,Reynah Morata,https://play-lh.googleusercontent.com/a-/ALV-U...,"Aku ngerasa ini apps kyknya palsu deh, kyk bin...",1,0,None,2024-05-03 17:16:43,Hai kak tidak perlu khawatir karena Bibit sdh ...,2024-05-03 17:58:02,None
2,6638614cdd520c9ba22ee66e,ad7e6c7f-3d78-4de8-ae75-00f8f3e0cd5e,Saputraaa_,https://play-lh.googleusercontent.com/a-/ALV-U...,MAAF SEBELUMNYA SAYA KASIH TAU JANGAN INVEST T...,1,0,3.79.0,2024-05-03 09:09:22,Hai kak. Dana penjualan dikirim ke rekening te...,2024-05-03 17:54:21,3.79.0
3,6638614cdd520c9ba22ee66f,fbf3b6f0-0c9a-4bf5-ae66-e20da7c296b9,only we,https://play-lh.googleusercontent.com/a/ACg8oc...,"sampah, udah jual di harga tinggi tapi karena ...",1,0,None,2024-05-03 07:50:00,"Hai kak, transaksi reksa dana tidak real time ...",2024-05-03 17:47:15,None
4,6638614cdd520c9ba22ee670,1e2e68eb-0581-4740-bc56-f3dd56605035,M Hafidz,https://play-lh.googleusercontent.com/a-/ALV-U...,Pencairan 300rb aja seperti 300jt. Maksimal 7 ...,1,0,3.79.0,2024-05-02 11:55:53,"Hai kak, dana penjualan dikirim Bank Kustodian...",2024-05-02 20:13:46,3.79.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9726,663864e9dd520c9ba22f284a,a9b816e8-9122-4131-80af-a4448479e58a,AL Kautsar A.M,https://play-lh.googleusercontent.com/a/ACg8oc...,"Aku suka ini aplikasi, next adain beli emas do...",5,1,None,2021-06-12 08:12:17,None,NaT,None
9727,663864e9dd520c9ba22f284b,04704246-5154-403a-9d03-cbd080e55b36,Eka Maelani,https://play-lh.googleusercontent.com/a/ACg8oc...,Aplikasi yang banyak manfaatnya. Terutama yang...,5,1,None,2020-12-31 17:43:10,None,NaT,None
9728,663864e9dd520c9ba22f284c,c2cbd006-3b7a-4e97-9478-fbe84ae4b2cf,Setyo Sabdono,https://play-lh.googleusercontent.com/a-/ALV-U...,Salah satu aplikasi APERD yang layak diintall ...,5,45,3.8.0,2021-04-06 22:11:23,None,NaT,3.8.0
9729,663864e9dd520c9ba22f284d,de64f08e-c76b-4980-8f5c-7716ac8ab98d,Sakina Tolemba,https://play-lh.googleusercontent.com/a-/ALV-U...,Saat Daftar Masukan Kode Refferal FARELTOLEMBA...,5,0,None,2023-01-01 15:18:20,None,NaT,None


In [16]:
duplicate_rows_content = df_bibit[df_bibit.duplicated(subset=['reviewId'])]
duplicate_rows_content

,_id,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion


In [17]:
df_bibit['score'].value_counts(dropna=False)

score
1    2943
4    2904
3    1393
5    1268
2    1223
Name: count, dtype: int64

In [19]:
df_bibit['year'] = df_bibit['at'].dt.year
df_bibit['year'].value_counts(dropna=False)

year
2022    3109
2021    2516
2023    1671
2024    1054
2020    1044
2019     337
Name: count, dtype: int64

## AJAIB

In [24]:
from google_play_scraper import Sort, reviews_all
import pymongo

def scrape_google_play_reviews(package_name, collection_name, num_loops):
    # Koneksi ke server MongoDB
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client["google_play_store_data"]
    collection = db[collection_name]

    # Looping sebanyak num_loops
    for _ in range(num_loops):
        # Looping untuk rating dari 1 hingga 5
        for rating in range(1, 6):
            # Looping untuk Sort.NEWEST dan Sort.MOST_RELEVANT
            for sort_option in [Sort.NEWEST, Sort.MOST_RELEVANT]:
                # Scrape semua ulasan yang tersedia
                result = reviews_all(
                    package_name,
                    sleep_milliseconds=0, # defaultnya 0
                    lang='id', # defaultnya 'en'
                    country='id', # defaultnya 'us'
                    count=500,
                    filter_score_with=rating,
                    sort=sort_option,
                )

                # Mencari konten ulasan yang sudah ada di MongoDB
                existing_contents = collection.distinct('reviewId')

                # Memfilter ulasan baru yang tidak duplikat berdasarkan konten
                new_unique_reviews = [review for review in result if review['reviewId'] not in existing_contents]

                # Menyimpan ulasan baru yang tidak duplikat
                if new_unique_reviews:
                    collection.insert_many(new_unique_reviews)
                    print(f"{len(new_unique_reviews)} ulasan baru dengan konten unik, rating {rating}, dan sort {sort_option.name} telah disimpan ke dalam MongoDB.")
                else:
                    print(f"Tidak ada ulasan baru dengan konten unik, rating {rating}, dan sort {sort_option.name} yang perlu disimpan.")

    print("Proses scraping selesai.")

# Input package name aplikasi yang akan di-scrape ulasannya
package_name = 'ajaib.co.id'
collection_name = 'raw_data_ajaib'

num_loops = 5  # Jumlah loop yang diinginkan

# Memanggil fungsi untuk melakukan scraping
scrape_google_play_reviews(package_name, collection_name, num_loops)


Tidak ada ulasan baru dengan konten unik, rating 1, dan sort NEWEST yang perlu disimpan.
Tidak ada ulasan baru dengan konten unik, rating 1, dan sort MOST_RELEVANT yang perlu disimpan.
Tidak ada ulasan baru dengan konten unik, rating 2, dan sort NEWEST yang perlu disimpan.
Tidak ada ulasan baru dengan konten unik, rating 2, dan sort MOST_RELEVANT yang perlu disimpan.
Tidak ada ulasan baru dengan konten unik, rating 3, dan sort NEWEST yang perlu disimpan.
Tidak ada ulasan baru dengan konten unik, rating 3, dan sort MOST_RELEVANT yang perlu disimpan.
Tidak ada ulasan baru dengan konten unik, rating 4, dan sort NEWEST yang perlu disimpan.
Tidak ada ulasan baru dengan konten unik, rating 4, dan sort MOST_RELEVANT yang perlu disimpan.
Tidak ada ulasan baru dengan konten unik, rating 5, dan sort NEWEST yang perlu disimpan.
Tidak ada ulasan baru dengan konten unik, rating 5, dan sort MOST_RELEVANT yang perlu disimpan.
Tidak ada ulasan baru dengan konten unik, rating 1, dan sort NEWEST yang pe

In [25]:
import pandas as pd
from pymongo import MongoClient

# Menghubungkan ke server MongoDB (secara lokal)
client = MongoClient('localhost', 27017)

# Memilih database yang akan digunakan (jika tidak ada, akan dibuat)
db = client['google_play_store_data']

# Memilih koleksi dalam database (jika tidak ada, akan dibuat)
collection = db['raw_data_ajaib']  # Mengganti 'nama_koleksi' dengan 'product_reviews'

# Mengambil semua dokumen dalam koleksi
cursor = collection.find()

# Mengonversi data ke dalam bentuk DataFrame
df_ajaib = pd.DataFrame(list(cursor))

# Menutup koneksi dengan server MongoDB
client.close()


df_ajaib


,_id,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,663862c1dd520c9ba22efdc2,3b9eb8fb-6589-4a84-a382-544876872ce0,ILHAM NUR CAHYA YUSUF,https://play-lh.googleusercontent.com/a-/ALV-U...,Ga bisa verifikasi jelek,1,0,2.41.0,2024-05-05 12:11:55,None,NaT,2.41.0
1,663862c1dd520c9ba22efdc3,d20b0988-a851-4184-b7cd-9cc7f8e7dedf,Sipri Leba,https://play-lh.googleusercontent.com/a/ACg8oc...,Selalu gagal dalam tanda tangan. Coba ulangi f...,1,0,2.41.0,2024-05-05 07:32:30,None,NaT,2.41.0
2,663862c1dd520c9ba22efdc4,735a6b26-faef-4a04-b9e7-4d660f325cdb,Nurmansyah Daeng,https://play-lh.googleusercontent.com/a/ACg8oc...,Lama banget selesai verifikasi nya sangat meng...,1,0,2.41.0,2024-05-05 06:58:43,None,NaT,2.41.0
3,663862c1dd520c9ba22efdc5,6b239e12-ab51-4ea0-b33d-1b82b5a5fbdf,MICHAEL HOSEA NUGRAHA,https://play-lh.googleusercontent.com/a-/ALV-U...,Kenapa pas jual saham selalu ga bisa bisa ya,1,0,2.41.0,2024-05-04 16:24:09,None,NaT,2.41.0
4,663862c1dd520c9ba22efdc6,b2780760-7dc6-4510-9573-4d1cebb2e688,sriiayu wi229,https://play-lh.googleusercontent.com/a-/ALV-U...,Aplikasi gatau lagi gangguan gatau apa atau ak...,1,0,2.41.0,2024-05-04 05:10:50,None,NaT,2.41.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9290,66386673dd520c9ba22f30c6,1e0d81bc-ba7c-4401-ad47-bd7446be781e,Ahmad Ridho Saputra,https://play-lh.googleusercontent.com/a/ACg8oc...,Aplikasi nya cukup baik untuk pemula belajar i...,4,0,None,2020-12-06 00:29:04,None,NaT,None
9291,66386673dd520c9ba22f30c7,d51e32ae-5461-4d66-ac59-45e9ac4d0dd1,Darajat ahmad hasan,https://play-lh.googleusercontent.com/a-/ALV-U...,Aplikasinya sangat berguna serta mudah dipahami,4,0,1.0.1D,2021-08-03 17:30:42,None,NaT,1.0.1D
9292,66386673dd520c9ba22f30c8,340e793b-a32f-42f6-90c6-5678c83cdb70,Fauzi Arif,https://play-lh.googleusercontent.com/a-/ALV-U...,"Tolong di tingkatkan lg..terutama,,agar tidak ...",4,0,1.0.1C,2021-07-15 09:37:46,None,NaT,1.0.1C
9293,66386673dd520c9ba22f30c9,722982c3-457b-4c28-809d-f61eda60d196,Dede Erwanda,https://play-lh.googleusercontent.com/a-/ALV-U...,So far so good. But can't order HMETD RI. Ulas...,4,56,1.0.1H,2021-09-30 10:28:32,"Halo Dede Erwanda, terimakasih atas rating yan...",2021-07-06 09:13:14,1.0.1H


In [26]:
duplicate_rows_content = df_ajaib[df_ajaib.duplicated(subset=['reviewId'])]
duplicate_rows_content

,_id,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion


In [27]:
df_ajaib['year'] = df_ajaib['at'].dt.year
df_ajaib['year'].value_counts(dropna=False)

year
2022    2670
2021    2669
2023    2171
2024     838
2020     644
2019     303
Name: count, dtype: int64

In [28]:
df_ajaib['score'].value_counts(dropna=False)

score
3    2900
4    1815
5    1711
2    1679
1    1190
Name: count, dtype: int64